# CS295/395: Secure Distributed Computation
## Homework 6

# Part 1

Consider the Danish Sugar Beet Auction, where MPC was used to set the market price for sugar beets. 

First, read the following references:
- [Wikipedia](https://en.wikipedia.org/wiki/Danish_Sugar_Beet_Auction)
- [Slides by Tomas Toft](https://csrc.nist.gov/csrc/media/events/meeting-on-privacy-enhancing-cryptography/documents/toft.pdf)
  - Page 6: Auction definition
  - Page 11: System architecture
  - Page 15-16: The protocol
- [The original paper](https://eprint.iacr.org/2008/068.pdf)

## Question 1

Describe the *threat model* of the approach for the sugar beet auction. Include:
- The threat model itself
- Limitations of the threat model
- Plausible attacks on the system under this threat model

YOUR ANSWER HERE

## Question 2

Describe a potential protocol for solving this problem. Justify your decisions.

YOUR ANSWER HERE

# Part 2

Consider the deployment of MPC to measure pay disparity by the Boston Women's Workforce Council.

First, read the following references:
- [Communications of the ACM Article](https://dl.acm.org/doi/10.1145/3029603) (then click "pdf")
- [Interview with Mayank Varia](https://blog.trustedci.org/2020/01/transition-to-practice-success-story.html)
- [Paper and video about this work](https://www.usenix.org/conference/soups2019/presentation/qin) (go to 4:30 in the video for a description of the system architecture)
- [BWWC's Page](https://thebwwc.org/mpc)
- [Data submission page for the system](https://100talent.org/) (you can't submit data without a valid ID)

## Question 3

Describe the *threat model* of the approach taken for the BWWC system. Include:
- The threat model itself
- Limitations of the threat model
- Plausible attacks on the system under this threat model

YOUR ANSWER HERE

## Question 4

Describe a potential protocol for solving this problem. Justify your decisions.

YOUR ANSWER HERE

## Question 5

Describe the similarities and differences between these two problems and the associated solutions.

YOUR ANSWER HERE